In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import joblib

def read_data(data_path):
    df1 = pd.read_csv(data_path)
    df1.replace(' ', '0', inplace=True)
    return df1


df_original = read_data('Data.csv')

rfc_best = joblib.load('./model.joblib')
df_sear = pd.read_csv('Singleuser.csv')
df_sear['Churn'] = 1
df_sear['cluster_number'] = 5

if ('customerID' in df_sear.columns):
    l = [df_sear.iloc[0:1], df_original.iloc[0:1000]]

else:
    searched = df_sear
    searched['customerID'] = 'Dummy'
    searched = searched[df_original.columns]
    l = [searched, df_original.iloc[0:1000]]

df_for_oht = pd.concat(l)

def convert_data(df_churn):
    df_churn = df_churn.reset_index(drop=True)
    empty_cols = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
                  'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
                  'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                  'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                  'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

    for i in empty_cols:
        df_churn[i] = df_churn[i].replace(" ", np.nan)

    df_churn.drop(['customerID', 'cluster_number'], axis=1, inplace=True)
    df_churn = df_churn.dropna()
    binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

    for i in binary_cols:
        df_churn[i] = df_churn[i].replace({"Yes": 1, "No": 0})

    # Encoding column 'gender'
    df_churn['gender'] = df_churn['gender'].replace({"Male": 1, "Female": 0})
    df_churn['Churn'] = df_churn['Churn'].replace({"Yes": 1, "No": 0})
    df_churn['PaymentMethod'] = df_churn['PaymentMethod'].replace({"Yes": 1, "No": 0})

    category_cols = ['PaymentMethod', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                     'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract']

    for cc in category_cols:
        dummies = pd.get_dummies(df_churn[cc], drop_first=False)
        dummies = dummies.add_prefix("{}#".format(cc))
        df_churn.drop(cc, axis=1, inplace=True)
        df_churn = df_churn.join(dummies)

    return df_churn


df4 = convert_data(df_for_oht)

l = list(df4.columns)
l = [l[-1]] + l[:-1]
df4 = df4[l]
to_pred = df4.iloc[0:1]

X1 = to_pred.loc[:, df4.columns != 'Churn']
y = to_pred.loc[:, df4.columns == 'Churn']
y_pred_prob = rfc_best.predict_proba(X1)
y_pred = rfc_best.predict(X1)
print("Churn Flag: {}\n \n Class Probabilities:\n'Churn=0'- {}\n'Churn=1' - {}".format(y_pred[0],y_pred_prob[0,0],y_pred_prob[0,1]))

Churn Flag: 0
 
 Class Probabilities:
'Churn=0'- 0.524548935073805
'Churn=1' - 0.47545106492619493


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
y_pred

array([0])

In [3]:
y_pred_prob

array([[0.52454894, 0.47545106]])